Reference:
http://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/

In [14]:
# LSTM for international airline passengers problem with regression framing
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return numpy.array(dataX), numpy.array(dataY)
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset
dataframe = pandas.read_csv('international-airline-passengers.csv', usecols=[1], engine='python', skipfooter=3)
dataset = dataframe.values
dataset = dataset.astype('float32')
print(dataset)

[[ 112.]
 [ 118.]
 [ 132.]
 [ 129.]
 [ 121.]
 [ 135.]
 [ 148.]
 [ 148.]
 [ 136.]
 [ 119.]
 [ 104.]
 [ 118.]
 [ 115.]
 [ 126.]
 [ 141.]
 [ 135.]
 [ 125.]
 [ 149.]
 [ 170.]
 [ 170.]
 [ 158.]
 [ 133.]
 [ 114.]
 [ 140.]
 [ 145.]
 [ 150.]
 [ 178.]
 [ 163.]
 [ 172.]
 [ 178.]
 [ 199.]
 [ 199.]
 [ 184.]
 [ 162.]
 [ 146.]
 [ 166.]
 [ 171.]
 [ 180.]
 [ 193.]
 [ 181.]
 [ 183.]
 [ 218.]
 [ 230.]
 [ 242.]
 [ 209.]
 [ 191.]
 [ 172.]
 [ 194.]
 [ 196.]
 [ 196.]
 [ 236.]
 [ 235.]
 [ 229.]
 [ 243.]
 [ 264.]
 [ 272.]
 [ 237.]
 [ 211.]
 [ 180.]
 [ 201.]
 [ 204.]
 [ 188.]
 [ 235.]
 [ 227.]
 [ 234.]
 [ 264.]
 [ 302.]
 [ 293.]
 [ 259.]
 [ 229.]
 [ 203.]
 [ 229.]
 [ 242.]
 [ 233.]
 [ 267.]
 [ 269.]
 [ 270.]
 [ 315.]
 [ 364.]
 [ 347.]
 [ 312.]
 [ 274.]
 [ 237.]
 [ 278.]
 [ 284.]
 [ 277.]
 [ 317.]
 [ 313.]
 [ 318.]
 [ 374.]
 [ 413.]
 [ 405.]
 [ 355.]
 [ 306.]
 [ 271.]
 [ 306.]
 [ 315.]
 [ 301.]
 [ 356.]
 [ 348.]
 [ 355.]
 [ 422.]
 [ 465.]
 [ 467.]
 [ 404.]
 [ 347.]
 [ 305.]
 [ 336.]
 [ 340.]
 [ 318.]
 [ 362.]
 

In [18]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
print(len(dataset))
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train))
print(len(test))

# reshape into X=t and Y=t+1
look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
print(trainX)

144
96
48
[[ 0.01544401  0.02702703  0.05405405]
 [ 0.02702703  0.05405405  0.04826255]
 [ 0.05405405  0.04826255  0.03281853]
 [ 0.04826255  0.03281853  0.05984557]
 [ 0.03281853  0.05984557  0.08494207]
 [ 0.05984557  0.08494207  0.08494207]
 [ 0.08494207  0.08494207  0.06177607]
 [ 0.08494207  0.06177607  0.02895753]
 [ 0.06177607  0.02895753  0.        ]
 [ 0.02895753  0.          0.02702703]
 [ 0.          0.02702703  0.02123553]
 [ 0.02702703  0.02123553  0.04247104]
 [ 0.02123553  0.04247104  0.07142857]
 [ 0.04247104  0.07142857  0.05984557]
 [ 0.07142857  0.05984557  0.04054055]
 [ 0.05984557  0.04054055  0.08687258]
 [ 0.04054055  0.08687258  0.12741312]
 [ 0.08687258  0.12741312  0.12741312]
 [ 0.12741312  0.12741312  0.10424709]
 [ 0.12741312  0.10424709  0.05598456]
 [ 0.10424709  0.05598456  0.01930502]
 [ 0.05598456  0.01930502  0.06949806]
 [ 0.01930502  0.06949806  0.07915059]
 [ 0.06949806  0.07915059  0.08880308]
 [ 0.07915059  0.08880308  0.14285713]
 [ 0.08880308  

In [13]:
# create and fit the LSTM network
batch_size = 1
model = Sequential()
model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, nb_epoch=100, batch_size=1, verbose=2)
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))
# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

Exception: Error when checking model input: expected lstm_input_2 to have 3 dimensions, but got array with shape (92, 3)